# Creating the containers for .NET notebooks for exploring dbatools

To run these notebooks, you will first have to have set up Jupyter Notebooks following the information here [https://sqldbawithabeard.com/2020/02/07/new-net-notebooks-are-here-powershell-7-notebooks-are-here/](https://sqldbawithabeard.com/2020/02/07/new-net-notebooks-are-here-powershell-7-notebooks-are-here/)

![dbatools](.\images\dbatools.png)

We have written a book which will give you a brilliant introduction to dbatools. It's called dbatools in a Month of Lunches and you can find it at [https://beard.media/book](https://beard.media/book)

# Setting up the containers for the rest of the dbatools notebooks

To be able to follow along with the rest of the notebooks, you will need to set up two containers.

This notebook will enable you to do that.

You will need to have Docker installed [Install Docker Desktop on Windows | Docker Documentation](https://docs.docker.com/docker-for-windows/install/). The image is based on the SQL Server 2019 image so you will need to have docker set to use Linux Containers. Right click on the Docker icon in the notification area and if it says "Switch to Linux Containers" click to switch and wait for Docker to restart. 

![Switch To Linux Containers](.\images\switchtolinuxcontainers.png)

You will be able to run all of the code in the notebooks by creating the folder, credential and containers in this notebook and then you can click on the play button in each code block to run the code. Note - There are a few code blocks with the results already included which should not be run. They are to show you the results of a command that cannot be run against containers (setting up configuration for domain accounts for example)

**You can also navigate through the Notebook using the arrow keys and press SHIFT + ENTER to run a cell**

## Create the folders and the credential

The code below will create a directory called dbatools-demo in your Documents folder and save a credential file for logging into the containers. You can alter the directory created by altering the `$FolderPath`  

The code has been collapsed but you can view it using the view menu in the lab or by clicking the 3 ellipses

In [ ]:
$NotebookDir = '' # add the directory that the notebook is in here 
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'

########################################################
Write-Output "Creating Directory $FolderPath"
if (Test-Path $FolderPath) {
    Write-Output "Path $FolderPath exists already"
}
else {
    $null = New-Item $FolderPath -ItemType Directory
}

Write-Output "Creating Directory $FolderPath\SQL1"
if (Test-Path "$FolderPath\SQL1") {
    Write-Output "Directory SQL1 exists already"
    Get-ChildItem "$FolderPath\SQL1" -Recurse | Remove-Item -Recurse -Force
}
else {
    $null = New-Item "$FolderPath\SQL1"-ItemType Directory
}
Write-Output "Creating File $FolderPath\SQL1\dummyfile.txt"
if (Test-Path "$FolderPath\SQL1\dummyfile.txt") {
    Write-Output "dummyfile.txt exists already"
}
else {
    $null = New-Item "$FolderPath\SQL1\dummyfile.txt" -ItemType file
}

Write-Output "Creating Directory $FolderPath\SQL2"
if (Test-Path "$FolderPath\SQL2") {
    Write-Output "Directory SQL2 exists already"
    Get-ChildItem "$FolderPath\SQL2" -Recurse | Remove-Item -Recurse -Force
}
else {
    $null = New-Item "$FolderPath\SQL2"-ItemType Directory
}
Write-Output "Creating File $FolderPath\SQL2\dummyfile.txt"
if (Test-Path "$FolderPath\SQL2\dummyfile.txt") {
    Write-Output "dummyfile.txt exists already"
}
else {
    $null = New-Item "$FolderPath\SQL2\dummyfile.txt" -ItemType file
}

Write-Output "Creating a credential file for the containers - Please don't do this in production"

$sqladminPassword = ConvertTo-SecureString 'dbatools.IO' -AsPlainText -Force 
$cred = New-Object System.Management.Automation.PSCredential ('sqladmin', $sqladminpassword)
$Cred | Export-Clixml -Path $FolderPath\sqladmin.cred
Write-Output "Credential file created"

Write-Output "Setting the docker-compose files values"

if ($NotebookDir -eq '') {
    try {
        $NotebookDir = Read-Host "Please enter path to the notebook. Azure Data Studio doesnt know where it is" -ErrorAction Stop
    }
    catch {
        Write-Warning "UH-OH - You have not set the `$NotebookDir variable to a directory we can find - Set it in the code please"
        Break
    }
}
if (-not (Test-Path $NotebookDir -ErrorAction SilentlyContinue)) {
    Write-Warning "UH-OH - You have not set the `$NotebookDir variable to a directory we can find  - Set it in the code please"
    Break
}

$dockercompose = (Get-Content $NotebookDir\\dockercompose.yml -ErrorAction Stop) -replace '__ReplaceME__' , $FolderPath
# $dockercompose
$dockercompose | Set-Content $NotebookDir\\docker-compose.yml
Set-Location $NotebookDir

Write-Output "Finished"

## Containers

We are going to create two SQL 2019 containers using the sqldbawithabeard\dbatoolsbeardsql01 [image from the Docker Hub](https://hub.docker.com/repository/docker/sqldbawithabeard/dbatoolsbeardsql01) and sqldbawithabeard\dbatoolsbeardsql02 [image from the Docker Hub](https://hub.docker.com/repository/docker/sqldbawithabeard/dbatoolsbeardsql02)

The first time it is going to pull the images from the Docker Hub. If you wish to do this first, you can run the cell below



In [ ]:
docker pull sqldbawithabeard/dbatoolsbeardsql01:v1
docker pull sqldbawithabeard/dbatoolsbeardsql02:v1

Now we can start the containers with the code below

In [ ]:
docker-compose up -d

All being well, you wil have something that looks like


![DockerCompose](.\images\containers.png )

If you get an error you might need to add the user you have shared your drives with Docker modify permissions to the \Documents\ directory in your user profile   
More details https://docs.docker.com/docker-for-windows/ or troubleshoot in the normal way


Now we can start exploring with dbatools :-)

If you have not installed dbatools, it can be got from the PowerShell Gallery using `Install-Module dbatools` the code below will check for the module and either install it in your user profile or update it and Import it

In [ ]:
if(Get-Module dbatools -ListAvailable){
Write-Output "Updating dbatools"
Update-Module dbatools
}else {
Write-Output "Installing dbatools in your user profile"
Install-Module dbatools -Scope CurrentUser
}
Import-Module dbatools

## Check connection
Now that is done, we can make a connection to our instances and see if we can connect to them.

We are going to use `Connect-DBaInstance` to do this and we will use the containers that we have created and because we need to use SQL Authentication, we will use a credential that we have saved to disk using `Export-CliXML` in one of the celss above.

It is **Important** to state that this is not a production secure solution and should not be used as a means of accessing any secure system.

In [ ]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'
$SqlInstances = 'localhost,15592', 'localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Write-Output " Creating connection to the containers"
try {
    $SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
    $SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
    Write-Output "We have a connection to the containers"

}
catch {
    Write-Output "You haven't got a connection to the containers - Either they are still upgrading in which case try again in 30 seconds or the containers have not come up correctly"
    Write-Output "Make sure the containers are running - the code is below in a block for you"
    Write-Output "docker ps -a"
    Write-Output "If they are read the logs - the code is below in a block for you"
    Write-Output "docker logs dbatools_SQL2019_1"
    Write-Output "docker logs dbatools_SQL2019-1_1"
}

If there are warnings above - Check the containers are running - Look at the status column

In [ ]:
docker ps -a

If there are warnings above and the containers are running - check the logs

In [ ]:
docker logs dbatools_SQL2019_1

If there are warnings above and the containers are running - check the logs

In [ ]:
docker logs dbatools_SQL2019_1

Now you can use these containers to run the rest of the notebooks - Don't forget to run the Clean Up Containers Notebook at the end to clean up